In [1]:
import os
os.environ['HF_HOME']="/GL8616/HF"
os.environ['HF_HUB_CACHE']="/GL8616/HF/HUB"
os.environ['HF_DATASETS_CACHE']="/GL8616/HF/DATA"

In [4]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [26]:
dataset['train'][130]

{'label': 1,
 'text': "The food is good. Unfortunately the service is very hit or miss. The main issue seems to be with the kitchen, the waiters and waitresses are often very apologetic for the long waits and it's pretty obvious that some of them avoid the tables after taking the initial order to avoid hearing complaints."}

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0,len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [24]:
df = pd.DataFrame(dataset['train'][[1,3,5,10]])
df

,label,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,3,Got a letter in the mail last week that said D...
2,4,Top notch doctor in a top notch practice. Can'...
3,0,Owning a driving range inside the city limits ...


In [15]:
show_random_elements(dataset["train"])

,label,text
0,2 star,"So I'm sure everyone has been to an Applebees in their lifetime. This particular location has terrible customer service! I've been there a few times hoping their customer service improves and only because it is a close location to where I live and for their happy hour. When you walk in you normally have to eye down the hosts or waiters to know that you are there waiting. Rarely is there anyone greeting you. They take a while with their food even when it's not busy, and their food is typically really salty. I wouldn't recommend this location, only if you're desperate for some cheap drinks during happy hour."
1,1 star,Worst customer service ever! Stay far far away. They try to take advantage whenever possible. All about how mush they can tip you off. Poor workmanship. Rude people. Lots if money.
2,4 stars,"Not Bad sushi...very cool decor, a little too air conditioned...Definately a young good looking college crowd...and then us late 20 and 30 somethings..\n\nA group of 12 of us went in tonight, a Tuesday, (we are all out of towners just here for work) and we got a table and served relatively quickly. \n\nWe ordered the Tuna Tar Tar, Spicy Tuna Roll, Rainbow Roll and some Nigiri...all I like is the Maguro combo's and they were great, nothing special, but tasty. At the other end of the table they got these deep fried rolls with cream cheese and all that I don't consider real sushi, they were probably good but I didn't try them.\n\nThe prices are ok, the 3 of us with all our sushi and $3.25 Miso soups and a few beers was only $77, maybe I'm just used to paying more in the San Fran area that I live... \n\nThe best part was the T-shirts all the waitresses wear..they say\n\n\""Will you still _____ me in the morning.\"" ....I guess you fill in the blank."
3,4 stars,Good shit. Good all around service. The food was prompt and the beer was ice cold. Fuck it don't need much more. Plus is happy hour till 6am. So can't complain.
4,3 stars,"I had the duck....excellent, excellent, excellent. My wife had the Gumbo, a complete miss. How do you miss on gumbo when your a southern restaurant is beyond me. Taking both of the plates into consideration and the fact that i thought it was overpriced, took it to a 3 star. \n\nSeparate note, the design and remodel of the old building is cool and worth checking out."
5,1 star,BLAND and GROSS.... Don't bother. Made me sick to my stomach
6,2 star,"I think my expectations were too high.. Way too high cause I'm left disappointed after my visit here. I never like writing negative reviews but I had to.. I just had to. \n\nFirst off, would it hurt for the workers to smile and greet customers? We were first timers and the menus were all over the place, I didn't even know where to look first. Seriously after 5 seconds of stepping inside, she goes \""What did you guys want?\"" Wait what, excuse me? Not even a \""Hello! How are you?\"" Or something.. Geez. \n\nHer monotonous voice and blank stare urged me to hurry up and figure out what I want so she doesn't get anymore pissed off. I opted for a Creamy Bun that everyone was hyping about and I got it with Cappuccino Crunch. It was very good, as I'm familiar with Thriftys ice cream. But dang, $5?! Idk about that.. Maybe $3.75 but not $5. The bf got a Honeydew Shaved Snow with Mochi, Popping Boba & Condensed Milk drizzle. $5 as well? Talk about ripoff!\n\nHere's the thing, were avid fans of Snowflake Shavery & shaved snow in general. Our cup was not filled with shaved snow/not filled up past the brim like the pictures on Yelp and Instagram, the toppings were tasty but a little more than a teaspoon would've been great and I didn't taste the condensed milk drizzle because well, it lacked of it.. We left totally unhappy, the girl was friendlier with the next customers who were young guys her age. Ugh!\n\nWhy us.. Why! We read and read the reviews here & I was so ready to write a similar great review. Not impressed at all. I'd rather drive to Chinatown

预处理数据
下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 map 方法，支持一次性在整个数据集上应用预处理函数。

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:20<00:00, 2423.07 examples/s]


In [32]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"After reading the other quality reviews, I had high hopes for Cyros as the wife and I have often driven by but opted for other battle-tested sushi options nearby. But, given a long holiday weekend and the chance to rid myself of a restaurant.com gift certificate, we headed over for dinner this week.\n\nGiven that it was a Thursday night, the place had a few tables full but the seating was open and we even had our choice of a regular table or a higher bar top. The crowd definitely felt \""Southpark\"" with the dyed blonde, botox-frozen ladies sipping glasses of wine while glaring at the pretty young woman in the other corner who's got a much older man wrapped around her arm. But, hey, that's the neighborhood, not Cyros' fault. Glad to know they've been accepted by the locals.\n\nOur downward spiral began with the service. Things never really clicked with our server, Gisele. She was slow to come to the table, almost invisible when it came time for refills, and reluctant to come back to check on the food she had just served. Now there were two other servers in the room who did a much better job- checking in with us and offering refills. Had they been our server, there'd at least be another star here as both seemed to genuinely be concerned with the customer experience both at our tables and those surrounding us.\n\nFor apps, I went with the seaweed salad and we had an order of endamame to share. Both were embarrassments. For $5, the seaweed salad was the smallest portion I had ever. I even took a picture and added it to this page (see above). It was literally served in what looked like a soy sauce dish. We also had to send back the endamame because it was cold. We saw another table do the same and overhead their server mention that they had a new guy in the back. Whoa, if we can't get steamed soybean pods correct then we're not ready for primetime.\n\nWe then waited (and waited) for our sushi. Nothing makes me more mad then seeing tables that arrived after us getting their meals ahead of us. I tried to reconcile this by thinking we ordered some fancy rolls, but the other tables did the same.\n\nWe went with the wasabi roll, the avante garde, and the nobu fire. While tasty, the rolls were pretty lose and did not hold up going from plate to soy sauce to mouth. They just seemed kind of sloppily made, almost lazy. It kinda fit the theme of the night.\n\nThe last insult was the bill. Now I fully accept that the certificate says 18% gratuity will be added to the whole bill. That's fine. But the final bill came back with a 20% tip. Here's the math- $52 total bill, $10.40 tip. 18% should have been $9.36. Not enough to fight over, but it's safe to say I won't be back- not where there's better quality and better service just around the corner.","[101, 1258, 3455, 1103, 1168, 3068, 3761, 117, 146, 1125, 1344, 7816, 1111, 27688, 5864, 1112, 1103, 1676, 1105, 146, 1138, 1510, 4940, 1118, 1133, 12243, 1111, 1168, 2321, 118, 7289, 28117, 5933, 6665, 2721, 119, 1252, 117, 1549, 170, 1263, 7946, 5138, 1105, 1103, 2640, 1106, 9297, 1991, 1104, 170, 4382, 119, 3254, 5309, 10890, 117, 1195, 2917, 1166, 1111, 4014, 1142, 1989, 119, 165, 183, 165, 183, 2349, 2109, 1179, 1115, 1122, 1108, 170, 9170, 1480, 117, 1103, 1282, 1125, 170, 1374, 7072, 1554, 1133, 1103, 11051, 1108, 1501, 1105, 1195, 1256, 1125, 1412, 3026, 1104, 170, 2366, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

数据抽样
使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

model_dir = "/GL8616/models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [10]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

In [1]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer
model_dir = "/GL8616/models/bert-base-cased-finetune-yelp"
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30,
                                 use_cpu=True #显存被占用，先使用CPU
                                 )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.325600,1.088172,0.554000
2,0.957100,1.109215,0.502000
3,0.644000,0.970028,0.602000


TrainOutput(global_step=189, training_loss=1.021029411800324, metrics={'train_runtime': 6565.4876, 'train_samples_per_second': 0.457, 'train_steps_per_second': 0.029, 'total_flos': 789354427392000.0, 'train_loss': 1.021029411800324, 'epoch': 3.0})

In [ ]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

In [ ]:
trainer.save_model(model_dir)

In [ ]:
trainer.save_state()